[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-ai-edge/model-explorer/blob/main/example_colabs/model_explorer_demo.ipynb) 

# Install Model Explorer and dependencies

In [ ]:
# Install tflite & model-explorer dependencies.
!pip install tflite
!pip install --no-deps ai-edge-model-explorer model-explorer-adapter

# Install kagglehub to load a model.
!pip install kagglehub --no-deps

# Download MobileNet v3 from Kaggle

In [ ]:
# TODO: Workaround until Eric's fixes
import kagglehub

# Download latest version
# TODO: Update this to use mobilebert or selected model.
path = kagglehub.model_download("google/mobilenet-v3/tfLite/large-075-224-classification")
model_path = f"{path}/1.tflite"

# Run the model with random test data

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Generate random input data.
for input_detail in input_details:
  input_shape = input_detail['shape']
  input_data = np.array(np.random.random_sample(input_shape), dtype=input_detail['dtype'])
  interpreter.set_tensor(input_detail['index'], input_data)

# Test the model on random input data.
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
for output_detail in output_details:
  print(f"Output for {output_detail['name']}")
  output_data = interpreter.get_tensor(output_detail['index'])
  print(output_data)

# Download the TF benchmark utility and run it

In [ ]:
%env MODEL_PATH=$model_path

!mkdir -p /tmp/data
%cd /tmp/data

# Download the tflite model benchmark binary.
!wget -nc https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model
!chmod +x /tmp/data/linux_x86-64_benchmark_model

# Run the benchmark locally only using CPU kernels.
# TODO: Update this once the feature is live.
# !./linux_x86-64_benchmark_model --graph=$MODEL_PATH --use_xnnpack=false --num_threads=4

# # Run the benchmark locally with XNNPACK kernels enabled.
# # TODO: Update this once the feature is live.
# !./linux_x86-64_benchmark_model --graph=$MODEL_PATH --use_xnnpack=true --num_threads=4

# TODO: Remove the below once the feature is live.
!wget -nc https://storage.googleapis.com/tfweb/model-explorer-demo/mv3-cpu-op-profile.json
!wget -nc https://storage.googleapis.com/tfweb/model-explorer-demo/mv3-xnnpack-op-profile.json
CPU_PROFILING_JSON_PATH="/tmp/data/mv3-cpu-op-profile.json"
XNNPACK_PROFILING_JSON_PATH="/tmp/data/mv3-xnnpack-op-profile.json"

# Visualize the model with the per op latency overlaid

In [ ]:
import model_explorer

config = model_explorer.config()
(config
 .add_model_from_path(model_path)
 .add_node_data_from_path(CPU_PROFILING_JSON_PATH)
 .add_node_data_from_path(XNNPACK_PROFILING_JSON_PATH))

model_explorer.visualize_from_config(config)